<h2> ======================================================</h2>
 <h1>MA477 - Theory and Applications of Data Science</h1> 
  <h1>Lesson 11: Logistic Regression </h1> 
 
 <h4>Dr. Valmir Bucaj</h4>
 United States Military Academy, West Point 
AY20-2
<h2>======================================================</h2>

<h2>Lecture Outline</h2>

<ul>
    <li>Why not Linear Regression?</li>
    <li> What is Logistic Regression?</li>
    <li>Estimating Regression Coefficients</li>
    <li>Logistic Regression with Multiple Features</li>
    <li> Logistic Regression with more than 2 response classes</li>
    
</ul>


<h2> Why not Linear Regression?</h2>

We will illustrate below that in the cases when the response variable is a qualitative variable, then linear regression may not be appropriate. So, why is that? 

Let's illustrate this using the `iris` dataset. In the iris dataset one is tasked to determine whether a plant is of type `setosa, versicolor` or `virginica` based on `sepal length, width` and `petal length, width`. 

So, the response variable $Y$ is clearly qualitative. Now, before we can fit a Linear Regression model (or most models for that matter) we need to first encode the outcomes using numbers. So, this is exactly where the dilemma lies. In other words, how do we encode a qualitative response variable with more than two possibilities? We will demonstrate shortly, that the model we obtain is highly sensitive to how you encode the qualitative variable. That is, if two different teams encoded the qualitative variables with more than two outcomes differently, they could end up with very different models which in turn would produce very different predictions on the new, previously unseen by the model, data. Obviously, this is a very undesirable outcome. 

For the purpose of this demonstration we will use the following two encodings for the response variable:

<b> Case 1</b>

$$Y_1=\begin{cases}
0 & \text{ setosa }\\
1 & \text{ versicolor }\\
2 & \text{ virginica }
\end{cases}
$$

<b> Case 2 </b>

$$Y_2=\begin{cases}
2 & \text{ setosa }\\
0 & \text{ versicolor }\\
1 & \text{ virginica }
\end{cases}
$$

We'll import our standard libraries as well as the `iris` dataset wich may be found in `sklearn.datasets`.


In [1]:
import numpy as np
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.datasets import load_iris

In [5]:
iris=load_iris()

In [7]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [8]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [10]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [29]:
df1=pd.DataFrame(iris.data,columns=iris.feature_names)

In [30]:
df1.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [31]:
df1['target']=iris.target

In [32]:
df1.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0




Recall that the current encodings of the response variable are as follows: 

$$Y_1=\begin{cases}
0 & \text{ setosa }\\
1 & \text{ versicolor }\\
2 & \text{ virginica }
\end{cases}
$$

In what follows, we will create a second dataframe `df2` with the response variable encoded as 

$$Y_1=\begin{cases}
2 & \text{ setosa }\\
0 & \text{ versicolor }\\
1 & \text{ virginica }
\end{cases}
$$

In [33]:
df2=df1.copy()
df2['target']=df2['target'].apply(lambda x: 2 if x==0 else 0 if x==1 else 1)

In [34]:
df2.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,2
1,4.9,3.0,1.4,0.2,2
2,4.7,3.2,1.3,0.2,2
3,4.6,3.1,1.5,0.2,2
4,5.0,3.6,1.4,0.2,2


Next, we will fit two Linear Regression models, and compare the coefficients of the model. 

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [36]:
lg1=LinearRegression()
lg2=LinearRegression()

In [37]:
X1=df1.drop("target",axis=1)
y1=df1['target']

X2=df2.drop("target",axis=1)
y2=df2['target']

In [42]:
X1_train,X1_test,y1_train,y1_test=train_test_split(X1,y1,test_size=0.1,random_state=1)

X2_train,X2_test,y2_train,y2_test=train_test_split(X2,y2,test_size=0.1,random_state=1)

In [43]:
X1_train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
42,4.4,3.2,1.3,0.2
92,5.8,2.6,4.0,1.2
66,5.6,3.0,4.5,1.5
31,5.4,3.4,1.5,0.4
35,5.0,3.2,1.2,0.2


In [44]:
X2_train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
42,4.4,3.2,1.3,0.2
92,5.8,2.6,4.0,1.2
66,5.6,3.0,4.5,1.5
31,5.4,3.4,1.5,0.4
35,5.0,3.2,1.2,0.2


One thing to note is that since we used the same random state for both splits, `X1_train` and `X2_train` are identical, as it is desired. The only thing that differs is `y's`, since we used different encodings. 

Now, we fit two linear models using these two datasets.

In [46]:
lg1.fit(X1_train,y1_train)

lg2.fit(X2_train,y2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Now, let's get the coefficients and compare them. 

In [48]:
coef1=lg1.coef_
coef2=lg2.coef_

In [51]:
df_coef=pd.DataFrame()
df_coef['coef1']=coef1
df_coef['coef2']=coef2

In [52]:
df_coef

,coef1,coef2
0,-0.143109,0.127436
1,-0.021897,0.661620
2,0.260573,-0.493013
3,0.570560,0.503313


Clearly, these coefficients are very different, and it is to be expected that they would produce different outcomes for the same input data. Let's test this. 

In [53]:
pred1=lg1.predict(X1_test)

pred2=lg2.predict(X2_test)

Next, let's put everything in a dataframe. Namely, the predictions of the two models, along with the actual classes of the plants.

In [59]:
df_pred=pd.DataFrame(index=y1_test.index)

df_pred['pred1']=pred1
df_pred['pred2']=pred2
df_pred['actual1']=y1_test
df_pred['actual2']=y2_test

In [61]:
df_pred

,pred1,pred2,actual1,actual2
14,-0.245633,2.408546,0,2
98,0.869925,0.892469,1,0
75,1.180283,0.875208,1,0
16,-0.046030,2.342771,0,2
131,1.840205,0.886133,2,1
56,1.408931,0.988222,1,0
141,1.831066,1.087474,2,1
44,0.155436,1.942570,0,2
29,0.033535,1.541865,0,2
120,1.985220,0.857828,2,1


In [62]:
df_pred['class name1']=df_pred['actual1'].apply(lambda x: 'setosa' if x==0 else 'versicolor' if x==1 else 'virginca')

In [64]:
df_pred['class name2']=df_pred['actual1'].apply(lambda x: 'setosa' if x==2 else 'versicolor' if x==0 else 'virginca')

In [65]:
df_pred

,pred1,pred2,actual1,actual2,class name,class name2
14,-0.245633,2.408546,0,2,setosa,versicolor
98,0.869925,0.892469,1,0,versicolor,virginca
75,1.180283,0.875208,1,0,versicolor,virginca
16,-0.046030,2.342771,0,2,setosa,versicolor
131,1.840205,0.886133,2,1,virginca,setosa
56,1.408931,0.988222,1,0,versicolor,virginca
141,1.831066,1.087474,2,1,virginca,setosa
44,0.155436,1.942570,0,2,setosa,versicolor
29,0.033535,1.541865,0,2,setosa,versicolor
120,1.985220,0.857828,2,1,virginca,setosa


As we can see, the two models could yield very different predictions. In fact, in this case, they don't match on a single output.

The other issue with this approach is the fact that the outcome of the linear regression cannot quite be interpreted as a probability since they may take values above one and below zero. 

<h2> Logistic Regression Model</h2>

For the remainder of this lesson, we will focus on the case of <b> binary</b> response variables. We will use the wisconsin breast cancer data set we previously used when we discussed KNN Classifier. 

Before we begin discussing the innerworkings of Logistic Regression, let's import the dataset.

In [66]:
from sklearn.datasets import load_breast_cancer

In [68]:
cancer=load_breast_cancer()

In [69]:
print(cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [70]:
df=pd.DataFrame(cancer.data,columns=cancer.feature_names)

In [71]:
df['target']=cancer.target

In [72]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


Recall that the target variable is a qualitative variable with two categories: <b> Malignant</b> and <b> Benign</b>. 

We will use the following encoding for the response variable:

$Y=\begin{cases} 0 & \text{ Malignant }\\
1 & \text{ Benign }
\end{cases}
$

<h3> How does Logistic Regression Work?</h3>

So, the response variable $Y$ can either be $0$ (Malignant) or $1$ (Benign). Instead of modeling directly the response variable $Y$, logistic regression, models instead the <b> probability </b> taht $Y$ belongs to a particular category. 

Specifically, if we denote by $X=(X_1,\dots, X_{30})$ all the $30$ cancer measurements, then logistic regression will instead model the probability that $Y=0$ or ($Y=$ Malignant) given the measurements $X$; that is 

$$p(X):= P\left(Y=0|X\right)$$

The values of $p(X)$ will be between $0$ and $1$. So, for example, we may predict <b> Malignant</b> if $p(X)>0.5$, or if we don't want to take a risk in misclassifying a malignant cell as benign, we can be more conservative and predict Malignant if $p(X)>0.1$ and predict <b> Benign</b> only if $p(X)\leq 0.1$. This is something that you as a Data Scientist have to decide depending on the goal at hand. 